In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_dynamics.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")

diff_numeric (generic function with 1 method)

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_n15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
@load "./data/quadstar_n15_funl" my_dict
Qnom = my_dict["Q"]
Knom = my_dict["K"]
Znom = my_dict["Z"];

In [4]:
using BenchmarkTools
function vech(A::AbstractMatrix{T},m::Int64) where T
    v = Vector{T}(undef, div(m*(m+1),2))
    # v = zeros(m*(m+1))
    k = 0
    @inbounds for j in 1:m
        @inbounds for i = 1:j
            v[k + i] = A[i,j]
        end
        k += j
    end
    return v
end
function inv_vech(v::AbstractVector{T},n::Int64) where {T}
    # M = zeros(T, n, n)
    M = Matrix{T}(undef, (n,n))
    @assert size(M, 2) == n
    @assert length(v) >= n * (n + 1) / 2
    k = 0
    @inbounds for j in 1:n
        @inbounds @simd for i = 1:j
            M[i,j] = v[k+i]
            M[j,i] = v[k+i]
        end
        k += j
    end
    return M
end

inv_vech (generic function with 1 method)

In [16]:
idx = 12
Q = Qnom[:,:,idx];
ix = size(Q,1)
i_upper_mat = triu(trues(size(Q)));

In [17]:
vec_Q = vec(Q)
vech_Q = vech(Q,ix);

In [18]:
@benchmark vech(Q,ix)

BenchmarkTools.Trial: 10000 samples with 927 evaluations.
 Range (min … max):  109.223 ns …   5.405 μs  ┊ GC (min … max): 0.00% … 93.57%
 Time  (median):     118.033 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   132.410 ns ± 115.173 ns  ┊ GC (mean ± σ):  9.10% ± 10.03%

  █▄▂▁                                                          ▁
  █████▆▆▇▅▁▃▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▃▃▃▄▃▄▁▄▅▅▅▇ █
  109 ns        Histogram: log(frequency) by time        940 ns <

 Memory estimate: 736 bytes, allocs estimate: 1.

In [19]:
@benchmark vec(Q)

BenchmarkTools.Trial: 10000 samples with 968 evaluations.
 Range (min … max):  79.718 ns …   3.125 μs  ┊ GC (min … max): 0.00% … 95.19%
 Time  (median):     80.708 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   87.555 ns ± 120.244 ns  ┊ GC (mean ± σ):  5.59% ±  3.96%

   ██▃▃▅▃▄▅▃▂▂▃▃▂▂▁▁▁▁                                         ▂
  ██████████████████████▇▇▇▇▇▆▇▇▅▆▅▆▅▆▅▅▄▅▅▅▃▅▄▅▅▄▄▄▅▃▅▅▄▄▅▅▄▅ █
  79.7 ns       Histogram: log(frequency) by time       105 ns <

 Memory estimate: 80 bytes, allocs estimate: 2.

In [20]:
@benchmark Q[i_upper_mat]

BenchmarkTools.Trial: 10000 samples with 807 evaluations.
 Range (min … max):  154.895 ns …  40.812 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     158.612 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   189.701 ns ± 525.528 ns  ┊ GC (mean ± σ):  6.48% ± 9.20%

  █▄▃▂                                                          ▁
  █████▇▆▆▆▄▄▆▄▃▄▃▃▃▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▃▁▁▃▁▁▁▃▁▁▁▃▁▃▁▃▃▄▁▄▄▁▁▄▄▆▆ █
  155 ns        Histogram: log(frequency) by time       1.13 μs <

 Memory estimate: 736 bytes, allocs estimate: 1.

In [ ]:
m  = triu(true(size(A)));
v  = A(m)

In [6]:
A = [1 2 3; 4 5 6; 7 8 9]

# Create the logical upper triangular mask
m = triu(trues(size(A)))

# Extract the elements of A using the mask
v = A[m]

println(v)

[1, 2, 5, 3, 6, 9]


In [ ]:
sum(abs.(inv_vech(vech_Q,ix) - Q))

In [ ]:
sum(abs.(reshape(vec_Q,(ix,ix))-Q))

In [ ]:
sum(abs.(Q - Q'))

In [ ]:
@benchmark inv_vech(vech_Q,ix)

In [ ]:
@benchmark reshape(vec_Q,(ix,ix))

In [ ]:
@benchmark vech(Q,ix)

In [ ]:
@benchmark vec(Q)

In [21]:
n = 3
Q = Symmetric(randn(n, n))
q = vec(Q)

9-element reshape(::Symmetric{Float64, Matrix{Float64}}, 9) with eltype Float64:
  0.6791524591268957
 -0.13658307669319839
  0.016217060946694662
 -0.13658307669319839
  1.9503317700341571
  0.46278801933497754
  0.016217060946694662
  0.46278801933497754
  0.3441433376320269

In [22]:
index_map = Dict()

# Fill index map for mapping elements of q to elements of q_u
k = 1
for j in 1:n
    for i in 1:j
        index_map[(i, j)] = k
        k += 1
    end
end

# Create the derivative matrix
D = zeros(n^2, length(index_map))
for j in 1:n
    for i in 1:n
        # Use symmetry to find the index
        idx = i <= j ? index_map[(i, j)] : index_map[(j, i)]
        D[(i-1)*n + j, idx] = 1
    end
end

# Output the matrix to verify
D

9×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [23]:
typeof(sparse(D))

SparseMatrixCSC{Float64, Int64}